In [2]:
### Use DataScience 2.0 Python 3

# Import raw data and store into S3

In [3]:
import boto3
import sagemaker
import pandas as pd

sess = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name
account_id = boto3.client("sts").get_caller_identity().get("Account")

sm = boto3.Session().client(service_name="sagemaker", region_name=region)

In [4]:
print(bucket)

sagemaker-us-east-1-212616788597


# Helper functions

In [5]:
import zipfile
import os

def unzip_file(zip_file_path, unzip_dir):
 
    if not os.path.exists(unzip_dir):
        os.makedirs(unzip_dir)

    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(unzip_dir)
        print(f"Unzipped {zip_file_path} to {unzip_dir}")

# Set S3 Source Location (Public S3 Bucket)

In [6]:
folder = "./data"
fake_csv = f"{folder}/Fake.csv.zip"
true_csv = f"{folder}/True.csv.zip"

unzip_file(fake_csv, folder)
unzip_file(true_csv, folder)

Unzipped ./data/Fake.csv.zip to ./data
Unzipped ./data/True.csv.zip to ./data


In [7]:
sagemkr_local_directory = 'data'
%store  sagemkr_local_directory

Stored 'sagemkr_local_directory' (str)


# Set S3 Destination Location (Our Private S3 Bucket)

In [8]:
s3_private_path_csv = "s3://{}/fake_news_dataset/csv".format(bucket)
print(s3_private_path_csv)

s3://sagemaker-us-east-1-212616788597/fake_news_dataset/csv


In [9]:
%store s3_private_path_csv

Stored 's3_private_path_csv' (str)


# Copy the data from local file system to our Private S3 Bucket in this Account

In [10]:
!aws s3 cp --recursive $folder $s3_private_path_csv/ --exclude "*.zip" --exclude "*.json"


upload: data/Fake.csv to s3://sagemaker-us-east-1-212616788597/fake_news_dataset/csv/Fake.csv
upload: data/True.csv to s3://sagemaker-us-east-1-212616788597/fake_news_dataset/csv/True.csv


In [11]:
print(s3_private_path_csv)

s3://sagemaker-us-east-1-212616788597/fake_news_dataset/csv


In [12]:
!aws s3 ls $s3_private_path_csv/

                           PRE .ipynb_checkpoints/
2024-05-21 05:23:20   62789876 Fake.csv
2024-05-21 04:55:54   23982555 Fake.csv.zip
2024-05-21 05:23:20   53582940 True.csv
2024-05-21 04:55:55   18993378 True.csv.zip
2024-05-21 04:55:55       5295 fake-and-real-news-dataset-metadata.json


# Panda's Dataframe

In [13]:
fake = pd.read_csv('data/Fake.csv')
fake.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [14]:
true = pd.read_csv('data/True.csv')
true.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


### Create Athena Databases

In [15]:
import awswrangler as wr

parquet_path = f"{s3_private_path_csv}-parquet-fake"

wr.s3.to_parquet(
    df=fake,
    path=parquet_path,
    dataset=True,
    mode='overwrite'
)

{'paths': ['s3://sagemaker-us-east-1-212616788597/fake_news_dataset/csv-parquet-fake/64d7674b37074112b46285d9d0b9541c.snappy.parquet'],
 'partitions_values': {}}

In [16]:
import awswrangler as wr

parquet_path = f"{s3_private_path_csv}-parquet-true"

wr.s3.to_parquet(
    df=true,
    path=parquet_path,
    dataset=True,
    mode='overwrite'
)

{'paths': ['s3://sagemaker-us-east-1-212616788597/fake_news_dataset/csv-parquet-true/c235afe3f7d84b95b92b347075e577bf.snappy.parquet'],
 'partitions_values': {}}

# SQL Queries

### Import PyAthena

In [17]:
from pyathena import connect

### Create an Athena Table

In [18]:
# Define the Athena database and table name
database = 'news'
fake_news_articles = 'fake_news_articles'
true_news_articles = 'true_news_articles'

fake_news_articles_s3 = f"{s3_private_path_csv}/Fake.csv"
true_news_articles_s3 = f"{s3_private_path_csv}/True.csv"


# Create the database if it doesn't exist
wr.athena.start_query_execution(
    sql=f"CREATE DATABASE IF NOT EXISTS {database}",
    database='default'
)

# Create or update the table in the specified database
wr.athena.start_query_execution(
    sql=f"""
    CREATE EXTERNAL TABLE IF NOT EXISTS {database}.{fake_news_articles} (
        title STRING,
        text STRING,
        subject STRING,
        date STRING
    )
    ROW FORMAT SERDE 'org.apache.hadoop.hive.serde2.lazy.LazySimpleSerDe'
    WITH SERDEPROPERTIES (
        'serialization.format' = '\t',
        'field.delim' = '\t'
    )
    STORED AS TEXTFILE
    LOCATION '{fake_news_articles_s3}'
    TBLPROPERTIES ('has_encrypted_data'='false');
    """,
    database=database
)

# Create or update the table in the specified database
wr.athena.start_query_execution(
    sql=f"""
    CREATE EXTERNAL TABLE IF NOT EXISTS {database}.{true_news_articles} (
        title STRING,
        text STRING,
        subject STRING,
        date STRING
    )
    ROW FORMAT SERDE 'org.apache.hadoop.hive.serde2.lazy.LazySimpleSerDe'
    WITH SERDEPROPERTIES (
        'serialization.format' = '\t',
        'field.delim' = '\t'
    )
    STORED AS TEXTFILE
    LOCATION '{true_news_articles_s3}'
    TBLPROPERTIES ('has_encrypted_data'='false');
    """,
    database=database
)

print("Athena database and table created successfully.")


Athena database and table created successfully.


## Queries
### Test the two tables

In [ ]:
import boto3
import sagemaker
import pandas as pd
import awswrangler as wr
from pyathena import connect

# Initialize SageMaker and boto3 session
sess = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name
account_id = boto3.client("sts").get_caller_identity().get("Account")

sm = boto3.Session().client(service_name="sagemaker", region_name=region)
print(bucket)

# Helper function to unzip files
import zipfile
import os

def unzip_file(zip_file_path, unzip_dir):
    if not os.path.exists(unzip_dir):
        os.makedirs(unzip_dir)

    with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
        zip_ref.extractall(unzip_dir)
        print(f"Unzipped {zip_file_path} to {unzip_dir}")

# Set S3 Source Location (Public S3 Bucket)
folder = "./data"
fake_csv = f"{folder}/Fake.csv.zip"
true_csv = f"{folder}/True.csv.zip"

unzip_file(fake_csv, folder)
unzip_file(true_csv, folder)

sagemkr_local_directory = 'data'
s3_private_path_csv = f"s3://{bucket}/fake_news_dataset/csv"
print(s3_private_path_csv)

# Copy the data from local file system to our Private S3 Bucket in this Account
os.system(f"aws s3 cp --recursive {folder} {s3_private_path_csv}/ --exclude '*.zip' --exclude '*.json'")

# Load data into Pandas DataFrames
fake = pd.read_csv('data/Fake.csv')
true = pd.read_csv('data/True.csv')

# Convert DataFrames to Parquet and upload to S3
parquet_path_fake = f"{s3_private_path_csv}-parquet-fake"
wr.s3.to_parquet(df=fake, path=parquet_path_fake, dataset=True, mode='overwrite')

parquet_path_true = f"{s3_private_path_csv}-parquet-true"
wr.s3.to_parquet(df=true, path=parquet_path_true, dataset=True, mode='overwrite')

# Define the Athena database and table names
database = 'news'
fake_news_articles = 'fake_news_articles'
true_news_articles = 'true_news_articles'

fake_news_articles_s3 = f"{parquet_path_fake}"
true_news_articles_s3 = f"{parquet_path_true}"

# Create the Athena database and tables
wr.athena.start_query_execution(
    sql=f"CREATE DATABASE IF NOT EXISTS {database}",
    database='default'
)

wr.athena.start_query_execution(
    sql=f"""
    CREATE EXTERNAL TABLE IF NOT EXISTS {database}.{fake_news_articles} (
        title STRING,
        text STRING,
        subject STRING,
        date STRING
    )
    STORED AS PARQUET
    LOCATION '{fake_news_articles_s3}'
    TBLPROPERTIES ('has_encrypted_data'='false');
    """,
    database=database
)

wr.athena.start_query_execution(
    sql=f"""
    CREATE EXTERNAL TABLE IF NOT EXISTS {database}.{true_news_articles} (
        title STRING,
        text STRING,
        subject STRING,
        date STRING
    )
    STORED AS PARQUET
    LOCATION '{true_news_articles_s3}'
    TBLPROPERTIES ('has_encrypted_data'='false');
    """,
    database=database
)

print("Athena database and tables created successfully.")

# Query the two tables
query_fake = f"""
SELECT title, text, subject, date
FROM {database}.{fake_news_articles}
LIMIT 10
"""

results_fake = wr.athena.read_sql_query(query_fake, database=database)
print("Fake News Articles:")
print(results_fake)

query_true = f"""
SELECT title, text, subject, date
FROM {database}.{true_news_articles}
LIMIT 10
"""

results_true = wr.athena.read_sql_query(query_true, database=database)
print("True News Articles:")
print(results_true)
